# Wikipedia Article Data

## Getting the Article, Population and Region Data

The first step is getting the data, which lives in several different places. We collect the data that lists the Wikipedia articles about US Cities and data for US state populations.

[The Wikipedia Category: Lists of cities in the United States by State](https://en.m.wikipedia.org/wiki/Category:Lists_of_cities_in_the_United_States_by_state) was crawled to generate a list of wikipedia article pages about US cities from each state. Location of the data in the homework folder: `input-data/us_cities_by_state_SEPT.2023.csv`

The US Census Bureau provides updated population estimates for every US state. The data for the population is taken from the State's [website](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html) and stored in the homework folder in `input-data/NST-EST2022-POP.xlsx`.

The region demarcation within the US is not one standardized and fixed thing. For this homework, we are using the regional and divisional agglomerations as defined by the US Census Bureau. The spreadsheet listing the states in each regional division is in the homework folder in `input-data/US States by Region - US Census Bureau.xlsx`


In [1]:
# ---------------------------- importing libraries --------------------------- #
import json, time
import requests
import pandas as pd

# to get a token for api
from dotenv import dotenv_values

In [2]:
# ----------------------- importing the input datasets ----------------------- #

df_cities_state = pd.read_csv("input-data/us_cities_by_state_SEPT.2023.csv")
df_population = pd.read_excel("input-data/NST-EST2022-POP.xlsx",
                              header=None,
                              skiprows=9,
                              names=['State','Base 2020', '2020 Estimate', '2021 Estimate', '2022 Estimate'],
                              skipfooter=5)
df_regions = pd.read_excel("input-data/US States by Region - US Census Bureau.xlsx")

## Checking the input data for any inconsistencies

Crawling Wikipedia categories to identify relevant page subsets can result in misleading and/or duplicate category labels. Naturally, the data crawl attempted to resolve these, but not all may have been caught.

### Inconsistencies in wikipedia data

In [3]:
df_cities_state.head()

,state,page_title,url
0,Alabama,"Abbeville, Alabama","https://en.wikipedia.org/wiki/Abbeville,_Alabama"
1,Alabama,"Adamsville, Alabama","https://en.wikipedia.org/wiki/Adamsville,_Alabama"
2,Alabama,"Addison, Alabama","https://en.wikipedia.org/wiki/Addison,_Alabama"
3,Alabama,"Akron, Alabama","https://en.wikipedia.org/wiki/Akron,_Alabama"
4,Alabama,"Alabaster, Alabama","https://en.wikipedia.org/wiki/Alabaster,_Alabama"


In [4]:
# -------------------------- checking for duplicates ------------------------- #

duplicate_records = df_cities_state[df_cities_state.duplicated(subset=['state', 'page_title', 'url'], keep = False)]

print(f'''There are a total of {duplicate_records.shape[0]} duplicate records.''')
print(f'Shape of the data: {df_cities_state.shape}')
print(f'Missing values: {df_cities_state.isnull().sum()}')

There are a total of 1260 duplicate records.
Shape of the data: (22157, 3)
Missing values: state         0
page_title    0
url           0
dtype: int64


There are 1260 records that have the same state, page_title and url values. As they are exactly the same, we drop the duplicate records and keep only the last one.

In [5]:
# dropping the duplicate records
df_cities_state = df_cities_state[~df_cities_state.duplicated(subset = ['state', 'page_title', 'url'], keep = 'last')]
print(f'Shape after dropping duplicates: {df_cities_state.shape}')

Shape after dropping duplicates: (21525, 3)


### Inconsistencies in population data

In [6]:
df_population.head()

,State,Base 2020,2020 Estimate,2021 Estimate,2022 Estimate
0,.Alabama,5024356.0,5031362.0,5049846.0,5074296.0
1,.Alaska,733378.0,732923.0,734182.0,733583.0
2,.Arizona,7151507.0,7179943.0,7264877.0,7359197.0
3,.Arkansas,3011555.0,3014195.0,3028122.0,3045637.0
4,.California,39538245.0,39501653.0,39142991.0,39029342.0


In [7]:
# removing the '.' that appears before each state name
df_population['State'] = df_population['State'].str.replace(".", "")
df_population.isnull().sum()

/var/folders/9r/7q8l4tjd43z8stsvm3s_5g400000gn/T/ipykernel_53271/2148530196.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_population['State'] = df_population['State'].str.replace(".", "")


State            1
Base 2020        1
2020 Estimate    1
2021 Estimate    1
2022 Estimate    1
dtype: int64

The output above shows that there is one null value in each column. This is because when you eyeball the dataset there is a blank row. As there is no data in any column, we drop the entire row from the data.

In [8]:
# ----------------- dropping the null values from the dataset ---------------- #

print(f'Original Shape of data: {df_population.shape}')
df_population.dropna(inplace = True)
print(f'Shape after dropping null values: {df_population.shape}')
print(f'Duplicate Values: {df_population.duplicated().sum()}')

Original Shape of data: (53, 5)
Shape after dropping null values: (52, 5)
Duplicate Values: 0


### Inconsistencies in regions data

In [9]:
df_regions.head()

,REGION,DIVISION,STATE
0,Northeast,NaN,NaN
1,NaN,New England,NaN
2,NaN,NaN,Connecticut
3,NaN,NaN,Maine
4,NaN,NaN,Massachusetts


The fix the structure of the df_regions we use the forward fill method to fill in the missing values.

In [10]:
# ------------------ restructing the data using forward fill ----------------- #

df_regions['REGION'].fillna(method = 'ffill', inplace = True)
df_regions['DIVISION'].fillna(method = 'ffill', inplace = True)
df_regions['STATE'].fillna(method = 'ffill', inplace = True)

# only keeping rows where state is not null
df_regions = df_regions[df_regions['STATE'].notnull()]
df_regions.reset_index(drop = True, inplace = True)

df_regions.head()

,REGION,DIVISION,STATE
0,Northeast,New England,Connecticut
1,Northeast,New England,Maine
2,Northeast,New England,Massachusetts
3,Northeast,New England,New Hampshire
4,Northeast,New England,Rhode Island


We have checked for consistences in all the three input data sets and handled them by dropping the duplicates and null values. We also fixed the structure of the regions data to not have any NaN values.

## Getting Article Quality Predictions

Now we need to get the predicted quality scores for each article in the Wikipedia dataset. We're using a machine learning system called ORES. This was originally an acronym for "Objective Revision Evaluation Service" but was simply renamed “ORES”. ORES is a machine learning tool that can provide estimates of Wikipedia article quality. The article quality estimates are, from best to worst:

- FA: Featured article
- GA: Good article
- B: B-class article
- C: C-class article
- Start: Start-class article
- Stub: Stub-class article

These were learned based on articles in Wikipedia that were peer-reviewed using the Wikipedia content assessment procedures.These quality classes are a sub-set of quality assessment categories developed by Wikipedia editors.

ORES requires a specific revision ID of a specific article to be able to make a label prediction. We are going to use the API: Info request to get a range of metadata on an article, including the most current revision ID of the article page. 

Putting this together, to get a Wikipedia page quality prediction from ORES for each politician’s article page you will need to: 

1. read each line of us_cities_by_state_SEPT.2023.csv
2. make a page info request to get the current page revision
3. make an ORES request using the page title and current revision id.

The homework folder contains example code in notebooks to illustrate making a page info request and making an ORES request. This sample code is licensed CC0 so feel free to reuse any of the code in either notebook without attribution. 

Note: It is possible that we will be unable to get a score for a particular article. If that happens, we are going to maintain a log of articles for which we were not able to retrieve an ORES score.

### Page Info API 

In [11]:
# -------- code snippet sourced from Dr. David W. McDonald's notebook -------- #

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<nsaumya@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": None,
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}

In [12]:
# -------- code snippet sourced from Dr. David W. McDonald's notebook -------- #

def request_pageinfo_per_article(article_title = None,
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT,
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):

    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [13]:
# --------------- defining the article titles for the function --------------- #

ARTICLE_TITLES = df_cities_state['page_title'].replace(" ", "_")
cities_info_filename = "intermediate-data/cities-info.json"

In [15]:
# -------------- running the api function for each article title ------------- #

cities_info = {}
for title in ARTICLE_TITLES[0:]:
  print(f"Getting page info data for: {title}")
  request_info = PAGEINFO_PARAMS_TEMPLATE.copy()
  request_info['titles'] = title
  cities_info[title] = request_pageinfo_per_article(request_template = request_info)

with open(cities_info_filename, "w") as cities_data:
  cities_data.write(json.dumps(cities_info, indent=4))

Getting page info data for: Abbeville, Alabama
Getting page info data for: Adamsville, Alabama
Getting page info data for: Addison, Alabama
Getting page info data for: Akron, Alabama
Getting page info data for: Alabaster, Alabama
Getting page info data for: Albertville, Alabama
Getting page info data for: Alexander City, Alabama
Getting page info data for: Aliceville, Alabama
Getting page info data for: Allgood, Alabama
Getting page info data for: Altoona, Alabama
Getting page info data for: Andalusia, Alabama
Getting page info data for: Anderson, Lauderdale County, Alabama
Getting page info data for: Anniston, Alabama
Getting page info data for: Arab, Alabama
Getting page info data for: Ardmore, Alabama
Getting page info data for: Argo, Alabama
Getting page info data for: Ariton, Alabama
Getting page info data for: Arley, Alabama
Getting page info data for: Ashford, Alabama
Getting page info data for: Ashland, Alabama
Getting page info data for: Ashville, Alabama
Getting page info dat

### Article Quality Prediction API

In [26]:
# read the JSON file saved from the previous step
with open("intermediate-data/cities-info.json", 'r') as json_file:
    cities_data = json.load(json_file)

In [34]:
# ------------------------- converting to a dataframe ------------------------ #

city_data_list = []
for city, city_info in cities_data.items():
    page_data = city_info["query"]["pages"]
    for page_id, page_info in page_data.items():
        city_data = {
            "pageid": page_info["pageid"],
            "ns": page_info["ns"],
            "title": page_info["title"],
            "contentmodel": page_info["contentmodel"],
            "pagelanguage": page_info["pagelanguage"],
            "pagelanguagehtmlcode": page_info["pagelanguagehtmlcode"],
            "pagelanguagedir": page_info["pagelanguagedir"],
            "touched": page_info["touched"],
            "lastrevid": page_info["lastrevid"],
            "length": page_info["length"],
            "fullurl": page_info["fullurl"],
            "editurl": page_info["editurl"],
            "canonicalurl": page_info["canonicalurl"]
        }
        city_data_list.append(city_data)

# creating a DataFrame
df = pd.DataFrame(city_data_list)

In [35]:
df.head()

,pageid,ns,title,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,touched,lastrevid,length,fullurl,editurl,canonicalurl
0,104730,0,"Abbeville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1171163550,24706,"https://en.wikipedia.org/wiki/Abbeville,_Alabama",https://en.wikipedia.org/w/index.php?title=Abb...,"https://en.wikipedia.org/wiki/Abbeville,_Alabama"
1,104761,0,"Adamsville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1177621427,18040,"https://en.wikipedia.org/wiki/Adamsville,_Alabama",https://en.wikipedia.org/w/index.php?title=Ada...,"https://en.wikipedia.org/wiki/Adamsville,_Alabama"
2,105188,0,"Addison, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1168359898,13309,"https://en.wikipedia.org/wiki/Addison,_Alabama",https://en.wikipedia.org/w/index.php?title=Add...,"https://en.wikipedia.org/wiki/Addison,_Alabama"
3,104726,0,"Akron, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1165909508,11710,"https://en.wikipedia.org/wiki/Akron,_Alabama",https://en.wikipedia.org/w/index.php?title=Akr...,"https://en.wikipedia.org/wiki/Akron,_Alabama"
4,105109,0,"Alabaster, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1179139816,20343,"https://en.wikipedia.org/wiki/Alabaster,_Alabama",https://en.wikipedia.org/w/index.php?title=Ala...,"https://en.wikipedia.org/wiki/Alabaster,_Alabama"


In [36]:
# ------------- .env for storing sensitive api token information ------------- #

wiki_tokens = dotenv_values(".env")
USERNAME = wiki_tokens['USERNAME']
ACCESS_TOKEN = wiki_tokens['ACCESS_TOKEN']

In [37]:
# -------- code snippet sourced from Dr. David W. McDonald's notebook -------- #

# The current LiftWing ORES API endpoint and prediction model
API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"

# The throttling rate is a function of the Access token that you are granted when you request the token. The constants
# come from dissecting the token and getting the rate limits from the granted token. An example of that is below.
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (60.0/5000.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
#
# Because all LiftWing API requests require some form of authentication, you need to provide your access token
# as part of the header too
REQUEST_HEADER_TEMPLATE = {
    'User-Agent': "nsaumya@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2023",
    'Content-Type': 'application/json',
    'Authorization': "Bearer {access_token}"
}

# This is a template for the parameters that we need to supply in the headers of an API request
REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address' : "saumya.nauni@gmail.com",
    'access_token'  : ACCESS_TOKEN
}

# This is a template of the data required as a payload when making a scoring request of the ORES model
ORES_REQUEST_DATA_TEMPLATE = {
    "lang":        "en",     # required that its english - we're scoring English Wikipedia revisions
    "rev_id":      "",       # this request requires a revision id
    "features":    True
}

In [38]:
# -------- code snippet sourced from Dr. David W. McDonald's notebook -------- #

def request_ores_score_per_article(article_revid = None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT,
                                   model_name = API_ORES_EN_QUALITY_MODEL,
                                   request_data = ORES_REQUEST_DATA_TEMPLATE,
                                   header_format = REQUEST_HEADER_TEMPLATE,
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):

    #    Make sure we have an article revision id, email and token
    #    This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid

    #   Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")

    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)

    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free data
        # source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
        prediction = json_response['enwiki']['scores'][str(article_revid)]['articlequality']['score']['prediction']
        prediction_score = json_response['enwiki']['scores'][str(article_revid)]['articlequality']['score']['probability'][prediction]
        if type(prediction_score) is not float:
            print("Prediction score is not a float")
    except Exception as e:
        print(e)
        prediction, prediction_score = (None, None)
    return (prediction, prediction_score)

In [40]:
# --- iterating through the dataframe to get the quality and article score --- #

# defining two column series
predictions = pd.Series(dtype='object')
prediction_scores = pd.Series(dtype='float64')

# iterating through the DataFrame and using the request ores function to get the quality score and article quality
for index, row in df.iterrows():
    print(row['title'])
    article_revid = row['lastrevid']
    prediction, prediction_score = request_ores_score_per_article(article_revid=article_revid)

    # appending the results to the Series
    predictions.at[index] = prediction
    prediction_scores.at[index] = prediction_score

# adding the series to the DataFrame
df['article_quality'] = predictions
df['quality_score'] = prediction_scores

Abbeville, Alabama
Adamsville, Alabama
Addison, Alabama
Akron, Alabama
Alabaster, Alabama
Albertville, Alabama
Alexander City, Alabama
Aliceville, Alabama
Allgood, Alabama
Altoona, Alabama
Andalusia, Alabama
Anderson, Lauderdale County, Alabama
Anniston, Alabama
Arab, Alabama
Ardmore, Alabama
Argo, Alabama
Ariton, Alabama
Arley, Alabama
Ashford, Alabama
Ashland, Alabama
Ashville, Alabama
Athens, Alabama
Atmore, Alabama
Attalla, Alabama
Auburn, Alabama
Autaugaville, Alabama
Avon, Alabama
Babbie, Alabama
Baileyton, Alabama
Bakerhill, Alabama
Banks, Alabama
Bay Minette, Alabama
Bayou La Batre, Alabama
Bear Creek, Alabama
Beatrice, Alabama
Beaverton, Alabama
Belk, Alabama
Benton, Alabama
Berlin, Alabama
Berry, Alabama
Bessemer, Alabama
Billingsley, Alabama
Birmingham, Alabama
Black, Alabama
Blountsville, Alabama
Blue Springs, Alabama
Boaz, Alabama
Boligee, Alabama
Bon Air, Alabama
Brantley, Alabama
Brent, Alabama
Brewton, Alabama
Bridgeport, Alabama
Brighton, Alabama
Brilliant, Alabama
Bro

In [41]:
df.head()

,pageid,ns,title,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,touched,lastrevid,length,fullurl,editurl,canonicalurl,article_quality,quality_score
0,104730,0,"Abbeville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1171163550,24706,"https://en.wikipedia.org/wiki/Abbeville,_Alabama",https://en.wikipedia.org/w/index.php?title=Abb...,"https://en.wikipedia.org/wiki/Abbeville,_Alabama",C,0.59792
1,104761,0,"Adamsville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1177621427,18040,"https://en.wikipedia.org/wiki/Adamsville,_Alabama",https://en.wikipedia.org/w/index.php?title=Ada...,"https://en.wikipedia.org/wiki/Adamsville,_Alabama",C,0.37707
2,105188,0,"Addison, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1168359898,13309,"https://en.wikipedia.org/wiki/Addison,_Alabama",https://en.wikipedia.org/w/index.php?title=Add...,"https://en.wikipedia.org/wiki/Addison,_Alabama",C,0.32446
3,104726,0,"Akron, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1165909508,11710,"https://en.wikipedia.org/wiki/Akron,_Alabama",https://en.wikipedia.org/w/index.php?title=Akr...,"https://en.wikipedia.org/wiki/Akron,_Alabama",GA,0.448584
4,105109,0,"Alabaster, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1179139816,20343,"https://en.wikipedia.org/wiki/Alabaster,_Alabama",https://en.wikipedia.org/w/index.php?title=Ala...,"https://en.wikipedia.org/wiki/Alabaster,_Alabama",C,0.646384


In [76]:
# -------------------------------- count of NA ------------------------------- #

df_failed = df[df['quality_score'].isna()]
print(f"Failed requests: {len(df_failed)}")

Failed requests: 3817


Because the number of failed requests is very high we run the ORES API call again only on the failed data rows to update them. The goal of this step is to retry the failed requests and check if they get the article and quality score.

In [77]:
# --- iterating on failed requests dataframe to get the quality and article score --- #

# defining a function that reruns the ORES API on the failed dataframe and updates the original one
def update_row(row):
    print(row['title'])
    article_revid = row['lastrevid']
    prediction, prediction_score = request_ores_score_per_article(article_revid = article_revid)
    row['article_quality'] = prediction
    row['quality_score'] = prediction_score
    return row

# using the apply method to update each row in df_failed and assign it back to the DataFrame
df_failed = df_failed.apply(update_row, axis=1)

# updating the original dataframe
for index, row in df_failed.iterrows():
    df.at[index, 'article_quality'] = row['article_quality']
    df.at[index, 'quality_score'] = row['quality_score']

Vidalia, Louisiana
Vienna, Louisiana
Ville Platte, Louisiana
Vinton, Louisiana
Vivian, Louisiana
Walker, Louisiana
Washington, Louisiana
Waterproof, Louisiana
Welsh, Louisiana
Westlake, Louisiana
West Monroe, Louisiana
Westwego, Louisiana
White Castle, Louisiana
Wilson, Louisiana
Winnfield, Louisiana
Winnsboro, Louisiana
Wisner, Louisiana
Woodworth, Louisiana
Youngsville, Louisiana
Zachary, Louisiana
Zwolle, Louisiana
Abbot, Maine
Acton, Maine
Addison, Maine
Albion, Maine
Alexander, Maine
Alfred, Maine
Allagash, Maine
Alna, Maine
Alton, Maine
Amherst, Maine
Amity, Maine
Andover, Maine
Anson, Maine
Appleton, Maine
Arrowsic, Maine
Arundel, Maine
Ashland, Maine
Athens, Maine
Auburn, Maine
Augusta, Maine
Aurora, Maine
Avon, Maine
Baileyville, Maine
Baldwin, Maine
Bangor, Maine
Bar Harbor, Maine
Baring Plantation, Maine
Bath, Maine
Beals, Maine
Beaver Cove, Maine
Beddington, Maine
Belfast, Maine
Belgrade, Maine
Belmont, Maine
Benton, Maine
Berwick, Maine
Bethel, Maine
Biddeford, Maine
Bingh

In [78]:
# reconfirming the number of failed requests (the numbers should be 0)
df_failed = df[df['quality_score'].isna()]
print(f"Failed requests: {len(df_failed)}")

Failed requests: 0


In [79]:
df_failed.isnull().sum()

pageid                  0.0
ns                      0.0
title                   0.0
contentmodel            0.0
pagelanguage            0.0
pagelanguagehtmlcode    0.0
pagelanguagedir         0.0
touched                 0.0
lastrevid               0.0
length                  0.0
fullurl                 0.0
editurl                 0.0
canonicalurl            0.0
article_quality         0.0
quality_score           0.0
dtype: float64

Now that there are no failed requests we proceed with saving the dataframe and performing the combining steps

In [80]:
# saving the dataframe
df.to_csv("intermediate-data/cities_article_quality.csv", index = False)

## Combining the Datasets

Some processing of the data will be necessary. In particular, merging the wikipedia data and population data together on state names. The combined dataset also requires labeling each state with it's US Census regional-division. The dataset `df_regions` represents regions, division and states hierarchically. We merge this dataset with the combined dataset. When merging the data there will be some non-states (Washington, DC or Puerto Rico), we ignore these non-states. We identify all areas for which there are no matches and output a list naming those areas, with each area on a separate line.

Lastly we consolidate the merged data into a single csv file: `data/wp_scored_city_articles_by_state.csv`

In [86]:
df = pd.read_csv("intermediate-data/cities_article_quality.csv")

In [87]:
# -------------- counting the count of classes in the dataframe -------------- #

df['article_quality'].value_counts()

C        12919
GA        4731
Start     2101
B          880
Stub       678
FA         210
Name: article_quality, dtype: int64

In [88]:
# -------------------------- modifying the dataframe ------------------------- #

# selecting only the columns that are required for the analysis
df = df[['title', 'lastrevid', 'article_quality']]
# renaming the columns for a more descriptive header
df.columns = ['article_title', 'revision_id', 'article_quality']

In [89]:
# ------------------- getting state names in the dataframe ------------------- #

# in order to merge the datasets we derive state name by joining on the page title found in the original dataset
df = df.merge(df_cities_state[['page_title', 'state']], left_on = 'article_title', right_on = 'page_title', how = 'left')
# dropping the page title column
df = df.drop(columns=['page_title'])

# -------------------- modifying the values in the columns ------------------- #

# replacing '_' with space in the state column (original form)
df['state'] = df['state'].str.replace("_", " ")

# replacing 'Georgia (U.S. state)' with 'Georgia' to match the state name in df_regions
df.loc[df['state'] == 'Georgia (U.S. state)', 'state'] = 'Georgia'

df.head()

,article_title,revision_id,article_quality,state
0,"Abbeville, Alabama",1171163550,C,Alabama
1,"Adamsville, Alabama",1177621427,C,Alabama
2,"Addison, Alabama",1168359898,C,Alabama
3,"Akron, Alabama",1165909508,GA,Alabama
4,"Alabaster, Alabama",1179139816,C,Alabama


In [90]:
# -------------- merging wikipedia data with the population data ------------- #

# keeping only state name and 2022 estimate columns required for the analysis
df_population = df_population[['State', '2022 Estimate']]

# merging df with df_population on the state name
df = df.merge(df_population, left_on = 'state', right_on = 'State', how = 'left')

df = df.drop(columns = ['State'])

df.head()

,article_title,revision_id,article_quality,state,2022 Estimate
0,"Abbeville, Alabama",1171163550,C,Alabama,5074296.0
1,"Adamsville, Alabama",1177621427,C,Alabama,5074296.0
2,"Addison, Alabama",1168359898,C,Alabama,5074296.0
3,"Akron, Alabama",1165909508,GA,Alabama,5074296.0
4,"Alabaster, Alabama",1179139816,C,Alabama,5074296.0


In [91]:
# ------------- merging the regions datafrane with the combined dataframe ------------ #

# mergeing the combined dataframes on the regions dataframe to get the region and division
df = df.merge(df_regions[['DIVISION','STATE']], left_on = 'state', right_on = 'STATE', how = 'left')

# droping the STATE column to avoid duplication
df = df.drop(columns=['STATE'])

In [92]:
df.head()

,article_title,revision_id,article_quality,state,2022 Estimate,DIVISION
0,"Abbeville, Alabama",1171163550,C,Alabama,5074296.0,East South Central
1,"Adamsville, Alabama",1177621427,C,Alabama,5074296.0,East South Central
2,"Addison, Alabama",1168359898,C,Alabama,5074296.0,East South Central
3,"Akron, Alabama",1165909508,GA,Alabama,5074296.0,East South Central
4,"Alabaster, Alabama",1179139816,C,Alabama,5074296.0,East South Central


In [97]:
# -------------------- renaming and reordering the columns ------------------- #

df = df.rename(columns={'DIVISION': 'regional_division', '2022 Estimate': 'population'})
df = df[['state', 'regional_division', 'population', 'article_title', 'revision_id', 'article_quality']]
df.head()

,state,regional_division,population,article_title,revision_id,article_quality
0,Alabama,East South Central,5074296.0,"Abbeville, Alabama",1171163550,C
1,Alabama,East South Central,5074296.0,"Adamsville, Alabama",1177621427,C
2,Alabama,East South Central,5074296.0,"Addison, Alabama",1168359898,C
3,Alabama,East South Central,5074296.0,"Akron, Alabama",1165909508,GA
4,Alabama,East South Central,5074296.0,"Alabaster, Alabama",1179139816,C


In [102]:
# ------------------ removing non-states from article_title ------------------ #

articles_to_remove = ['Federal Information Processing Standards',
 'American National Standards Institute',
 'Geographic Names Information System',
 'Wikipedia:Citation needed',
 '2020 United States census',
 '2010 United States census',
 'County (United States)',
 'Population',
 'Square mile']
for index, row in df.iterrows():
    if row['article_title'] in articles_to_remove:
        df.drop(index, inplace=True)
        print(f"Dropped {row['article_title']}")

Dropped 2010 United States census
Dropped 2010 United States census
Dropped 2010 United States census
Dropped 2010 United States census
Dropped 2020 United States census
Dropped 2020 United States census
Dropped 2020 United States census
Dropped 2020 United States census
Dropped County (United States)
Dropped County (United States)
Dropped County (United States)
Dropped County (United States)
Dropped County (United States)
Dropped County (United States)
Dropped Population
Dropped Population
Dropped Population
Dropped Population
Dropped Square mile
Dropped Federal Information Processing Standards
Dropped American National Standards Institute
Dropped Geographic Names Information System
Dropped Wikipedia:Citation needed
Dropped Wikipedia:Citation needed


In [104]:
# ----------------------- checking for duplicate values ---------------------- #

duplicates = df[df.duplicated()]
sum_of_duplicates = duplicates.shape[0]
sum_of_duplicates

4088

As there are a large number of duplicates, we drop the duplicate values that have the same values across all columns

In [106]:
# ------------------------- dropping duplicate values ------------------------ #

df = df.drop_duplicates()

#rechecking duplicates
duplicates = df[df.duplicated()]
duplicates.shape[0]

0

In [107]:
df.head()

,state,regional_division,population,article_title,revision_id,article_quality
0,Alabama,East South Central,5074296.0,"Abbeville, Alabama",1171163550,C
1,Alabama,East South Central,5074296.0,"Adamsville, Alabama",1177621427,C
2,Alabama,East South Central,5074296.0,"Addison, Alabama",1168359898,C
3,Alabama,East South Central,5074296.0,"Akron, Alabama",1165909508,GA
4,Alabama,East South Central,5074296.0,"Alabaster, Alabama",1179139816,C


In [108]:
df.to_csv("data/wp_scored_city_articles_by_state.csv", index=False)